In [67]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [68]:
cd /gdrive/MyDrive/4th Sem/Thesis/Hindi Hostility Detection/Notebooks/Models/Our Baseline


/gdrive/.shortcut-targets-by-id/13B8T3YvpbNgM6K13VzUAYOijsibQvWUN/Thesis/Hindi Hostility Detection/Notebooks/Models/Our Baseline


In [69]:
ls

'Dataset Embeddings'/   Models/


In [70]:
import pandas as pd
import numpy as np
import pickle
import copy
import re
import matplotlib.pyplot as plt
from sklearn.externals import joblib

from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [72]:
import sklearn
print(sklearn.__version__)

0.22.2.post1


In [ ]:
pd.options.display.max_colwidth = 0

### Preprocess Data

In [ ]:
def preprocess(text):
    text = text.lower()
    url = r'http\S+'
    text = re.sub(url, ' ', text, flags=re.MULTILINE)
    emoji = re.compile("["         u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002500-\U00002BEF"  # chinese char
                                   u"\U00002702-\U000027B0"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u"\U00010000-\U0010ffff"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u200d"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\ufe0f"  # dingbats
                                   u"\u3030"
                                   "]+", flags=re.UNICODE)
    text =  emoji.sub(r'', text)
    text = ' '.join([word[1:] if word[0] == '#' else word for word in text.split()])

    text = text.split()
    exclude_char_list = ['@','rt','|','%', '[', '(', '{', '}', ']', ')','url','URL']
    text = [word for word in text if all(ch not in word for ch in exclude_char_list)]
    text = ' '.join(text)
    return text

### Extracting mBERT Embeddings

In [ ]:
!pip install flair

     |████████████████████████████████| 450kB 4.2MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 
     |████████████████████████████████| 1.1MB 17.8MB/s 
     |████████████████████████████████| 1.3MB 23.5MB/s 
     |████████████████████████████████| 983kB 33.4MB/s 
     |████████████████████████████████| 798kB 40.7MB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
     |████████████████████████████████| 890kB 44.3MB/s 
     |████████████████████████████████| 2.9MB 49.1MB/s 
  Created wheel for sqlitedict: filename=sqlitedict-1.7.0-cp36-none-any.whl size=14376 sha256=71a910ae762d50abae2d3bf1325fdc3c7738e00dfb27a83f67c9e4d797b8dd2c
  Stored in directory: /root/.cache/pip/wheels/cf/c6/4f/2c64a43f041415eb8b8740bd80e15e92f0d46c5e464d8e4b9b
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45613 sha256=cda1889eceecf69db94d90a1635d22aad9c5899faaf4091dc760c470263a61cb
  Stored in directory: /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34

In [ ]:
from flair.data import Sentence
from flair.embeddings import BertEmbeddings
embeddings = BertEmbeddings('bert-base-multilingual-uncased', layers='-1') # use only last layers
import torch

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
def get_mbert_embeddings(text):

    sentence = Sentence(text)
    embeddings.embed(sentence)
    for token in sentence:
        z = token.embedding.size()[0]
        w = torch.zeros(0,z)
        # storing word Embeddings of each word in a sentence #
        w = torch.cat((w,token.embedding.view(-1,z)),0)
    # storing sentence Embeddings (mean of embeddings of all words)   #
    s = torch.zeros(0,z)
    s = torch.cat((s, w.mean(dim = 0).view(-1, z)),0)
    s = s.numpy()
    s = s.reshape(-1)
    
    return s

### Loading Fine-Grained Models

In [ ]:
hostlity_clf    = joblib.load("Models/hostility_clf.pkl")
defamation_clf  = joblib.load("Models/svm_defamation_clf.pkl")
fake_clf        = joblib.load("Models/svm_fake_clf.pkl")
hate_clf        = joblib.load("Models/svm_hate_clf.pkl")
offensive_clf   = joblib.load("Models/svm_offensive_clf.pkl")

### Input Text

In [ ]:
# text = "Hello hi how are you?"

# text = "मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है 🙏 जय श्री राम 🙏"

text = "कोई भी कांग्रेसी, ऊंची छत पर, रेलवे लाइन पर, ऊंची बिल्डिंग पर, एकांत जगह पर, कुए के पास दिखाई दे, तुरंत पुलिस को सूचित करें,🙏😂👍 विलुप्त होती हुई प्रजातियो को बचाना हमारा फर्ज है।"

### Prediction

In [ ]:
def predict(text):
    text = preprocess(text)
    embedding = get_mbert_embeddings(text)

    coarse_op       = hostlity_clf.predict([embedding,embedding])
    defamation_op   = defamation_clf.predict([embedding,embedding])
    fake_op         = fake_clf.predict([embedding,embedding])
    hate_op         = hate_clf.predict([embedding,embedding])
    offensive_op    = offensive_clf.predict([embedding,embedding])
    
    print(coarse_op)
    print(defamation_op)
    print(fake_op)
    print(hate_op)
    print(offensive_op)

    final_label = []

    if(defamation_op[0]==1):
        final_label.append("Defamation")

    if(fake_op[0]==1):
        final_label.append("Fake")
    
    if(hate_op[0]==1):
        final_label.append("Hate")

    if(offensive_op[0]==1):
        final_label.append("Offensive")
    
    if(len(final_label)==0):
        final_label.append("Non-Hostile")

    print(f'The Post "{text}" is --> {final_label}')


In [ ]:
predict(text)

[1. 1.]
[0. 0.]
[1. 1.]
[1. 1.]
[0. 0.]
The Post "कोई भी कांग्रेसी, ऊंची छत पर, रेलवे लाइन पर, ऊंची बिल्डिंग पर, एकांत जगह पर, कुए के पास दिखाई दे, तुरंत पुलिस को सूचित करें, विलुप्त होती हुई प्रजातियो को बचाना हमारा फर्ज है।" is --> ['Fake', 'Hate']
